In [ ]:
import numpy as np
import random
import math
import time

In [ ]:
#define amount of rows and columns
ROWS = 6
COLUMNS = 7
WINDOW_LENGTH = 4
COLUMN_COUNT=7
ROW_COUNT=6
EMPTY=0

In [ ]:
#create the board
def create_board(): 
    board = np.zeros((ROWS,COLUMNS))
    return board

In [ ]:
#check the drop is valid i.e. the row is not full
def valid_drop(board, col):
    if col>= 0 and col<=6:
        return board[ROWS-1][col] == 0
    else:
        return False

In [ ]:
#find where the counter will drop to
def counter_drop_location(board, col):
    for r in range(ROWS):
        if board[r][col] == 0:
            return r

In [ ]:
def print_board(board):
    print(np.flip(board,0))
    

In [ ]:
#choose where to drop counter
def drop_counter(board, row, col, counter):
    board[row][col] = counter

In [ ]:
#kieth galli minimax AI
def winning_move(board, piece):
	# Check horizontal locations for win
	for c in range(COLUMN_COUNT-3):
		for r in range(ROW_COUNT):
			if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
				return True

	# Check vertical locations for win
	for c in range(COLUMN_COUNT):
		for r in range(ROW_COUNT-3):
			if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
				return True

	# Check positively sloped diaganols
	for c in range(COLUMN_COUNT-3):
		for r in range(ROW_COUNT-3):
			if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
				return True

	# Check negatively sloped diaganols
	for c in range(COLUMN_COUNT-3):
		for r in range(3, ROW_COUNT):
			if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
				return True

def evaluate_window(window, piece):
	score = 0
	opp_piece = 1
	if piece == 1:
		opp_piece = 2

	if window.count(piece) == 4:
		score += 100
	elif window.count(piece) == 3 and window.count(EMPTY) == 1:
		score += 5
	elif window.count(piece) == 2 and window.count(EMPTY) == 2:
		score += 2

	if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
		score -= 4

	return score

def score_position(board, piece):
	score = 0

	## Score center column
	center_array = [int(i) for i in list(board[:, COLUMN_COUNT//2])]
	center_count = center_array.count(piece)
	score += center_count * 3

	## Score Horizontal
	for r in range(ROW_COUNT):
		row_array = [int(i) for i in list(board[r,:])]
		for c in range(COLUMN_COUNT-3):
			window = row_array[c:c+WINDOW_LENGTH]
			score += evaluate_window(window, piece)

	## Score Vertical
	for c in range(COLUMN_COUNT):
		col_array = [int(i) for i in list(board[:,c])]
		for r in range(ROW_COUNT-3):
			window = col_array[r:r+WINDOW_LENGTH]
			score += evaluate_window(window, piece)

	## Score posiive sloped diagonal
	for r in range(ROW_COUNT-3):
		for c in range(COLUMN_COUNT-3):
			window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
			score += evaluate_window(window, piece)

	for r in range(ROW_COUNT-3):
		for c in range(COLUMN_COUNT-3):
			window = [board[r+3-i][c+i] for i in range(WINDOW_LENGTH)]
			score += evaluate_window(window, piece)

	return score

def is_terminal_node(board):
	return winning_move(board, 1) or winning_move(board, 2) or len(get_valid_locations(board)) == 0

def minimax(board, depth, alpha, beta, maximizingPlayer):
	valid_locations = get_valid_locations(board)
	is_terminal = is_terminal_node(board)
	if depth == 0 or is_terminal:
		if is_terminal:
			if winning_move(board, 2):
				return (None, 100000000000000)
			elif winning_move(board, 1):
				return (None, -10000000000000)
			else: # Game is over, no more valid moves
				return (None, 0)
		else: # Depth is zero
			return (None, score_position(board, 2))
	if maximizingPlayer:
		value = -math.inf
		column = random.choice(valid_locations)
		for col in valid_locations:
			row = counter_drop_location(board, col)
			b_copy = board.copy()
			drop_counter(b_copy, row, col, 2)
			new_score = minimax(b_copy, depth-1, alpha, beta, False)[1]
			if new_score > value:
				value = new_score
				column = col
			alpha = max(alpha, value)
			if alpha >= beta:
				break
		return column, value

	else: # Minimizing player
		value = math.inf
		column = random.choice(valid_locations)
		for col in valid_locations:
			row = counter_drop_location(board, col)
			b_copy = board.copy()
			drop_counter(b_copy, row, col, 1)
			new_score = minimax(b_copy, depth-1, alpha, beta, True)[1]
			if new_score < value:
				value = new_score
				column = col
			beta = min(beta, value)
			if alpha >= beta:
				break
		return column, value


In [ ]:
def get_valid_locations(board):
    valid_locations = []
    for col in range(COLUMNS):
        if valid_drop(board, col):
            valid_locations.append(col)
    return valid_locations


In [ ]:
def dominating_ai_twoturns(board, counter):
    winning_cols=[0 for n in get_valid_locations(board)]
    for n in range(len(winning_cols)):
        dummy_board=board.copy()
        col=get_valid_locations(board)[n]
        row=counter_drop_location(dummy_board, col)
        drop_counter(dummy_board, row,col, counter)
        if game_win(dummy_board, counter, row, col):
            return col
        else:
            for col2 in get_valid_locations(dummy_board):
                dummy_boardb=dummy_board.copy()
                print_board(dummy_boardb)
                row2=counter_drop_location(dummy_boardb, col2)
                drop_counter(dummy_boardb, row2,col2, counter%2 + 1)
                for col3 in get_valid_locations(dummy_boardb):
                    dummy_boardc=dummy_boardb.copy()
                    row3=counter_drop_location(dummy_boardc, col3)
                    drop_counter(dummy_boardc, row3,col3, counter)
                    if game_win(dummy_boardc, counter, row3, col3):
                        winning_cols[n]+=1
    if sum(winning_cols)!=0:
        max_value = max(winning_cols)
        max_index = winning_cols.index(max_value)
        return get_valid_locations(board)[max_index]
                
    else:
        return random.choice(get_valid_locations(board))
            
    

In [ ]:
def dominated_ai_2turn(board, counter):
    posscol=[]
    for col in get_valid_locations(board):
        posscol.append(col)
        dummy_board=board.copy()
        row=counter_drop_location(dummy_board, col)
        drop_counter(dummy_board, row,col, counter)
        for col2 in get_valid_locations(dummy_board):
            dummy_boardb=dummy_board.copy()
            print_board(dummy_boardb)
            row2=counter_drop_location(dummy_boardb, col2)
            drop_counter(dummy_boardb, row2,col2, counter%2 + 1)
            if game_win(dummy_boardb,counter%2 + 1, row2, col2):
                if col in posscol:
                    posscol.remove(col)
    if len(posscol)==0:
        return np.random.randint(7)
    else:
        return random.choice(posscol)
    

In [1]:
def two_turn_ai(board, counter):
    posscols=[]
    winning_cols=[0 for n in get_valid_locations(board)]
    most_winning_col=-1
    for n in range(len(winning_cols)):
        col=get_valid_locations(board)[n]
        posscols.append(col)
        dummy_board=board.copy()
        row=counter_drop_location(dummy_board, col)
        drop_counter(dummy_board, row,col, counter)
        if winning_move(dummy_board, counter):
            return col
        else:
            dummy_boardb=dummy_board.copy()
            for col2 in get_valid_locations(dummy_board):
                dummy_boardb=dummy_boardb.copy()
                row2=counter_drop_location(dummy_boardb, col2)
                drop_counter(dummy_boardb, row2,col2, counter%2 + 1)
                if winning_move(dummy_boardb,counter%2 + 1):
                    if col in posscols:
                        posscols.remove(col)
            for col3 in posscols and get_valid_locations(dummy_boardb):
                dummy_boardc=dummy_boardb.copy()
                row3=counter_drop_location(dummy_boardc, col3)
                drop_counter(dummy_boardc, row3,col3, counter)
                if winning_move(dummy_boardc, counter):
                    winning_cols[n]+=1
    if sum(winning_cols)!=0:
        max_value = max(winning_cols)
        max_index = winning_cols.index(max_value)
        most_winning_col=get_valid_locations(board)[max_index]
    if most_winning_col in posscols:
        return most_winning_col
    elif len(posscols)!=0:
        return random.choice(posscols)
    else:
        return random.choice(get_valid_locations(board))
            

In [ ]:
def two_turn_ai_MCTS(board, counter):
    winning_col=9
    posscols=[]
    for i in get_valid_locations(board):
        posscols.append(i)
        dummy_board=board.copy()
        col=i
        row=counter_drop_location(dummy_board, col)
        drop_counter(dummy_board, row,col, counter)
        if winning_move(dummy_board, counter):
            winning_col = i
            return [winning_col]
        else:
            for j in get_valid_locations(dummy_board):
                dummy_boardb=dummy_board.copy()
                col2=j
                row2=counter_drop_location(dummy_boardb, col2)
                drop_counter(dummy_boardb, row2,col2, counter%2 + 1)
                if winning_move(dummy_boardb,counter%2 + 1):
                    if i in posscols:
                        posscols.remove(i)
    if len(posscols)!=0:
        return posscols
    else:
        return get_valid_locations(board)
            

In [ ]:
def montecarlo(board, counter, max_time, turn, combinatorial):
    if len(two_turn_ai_MCTS(board, counter))==1:
        return two_turn_ai_MCTS(board, counter)[0]
    scores=[[0]]
    visits=[[0]]
    expand(visits, scores, board, counter, [0])
    start=time.time()
    while time.time()<start+max_time:
        child_node=False
        state=0
        dummy_board=board.copy()
        play_counter=counter
        traverse_scores=scores.copy()
        traverse_visits=visits.copy()
        state_save=[0]
        won_game=False
        while not child_node:
            next_state=best_ucb1(traverse_scores[state], traverse_visits[state], 3)
            col=get_valid_locations(dummy_board)[next_state-1]
            row=counter_drop_location(dummy_board, col)
            drop_counter(dummy_board, row, col, play_counter)
            traverse_scores=traverse_scores[state]
            traverse_visits=traverse_visits[state]
            state=next_state
            state_save.append(state)
            play_counter=play_counter%2+1
            if len(traverse_visits[state])==1:
                    child_node=True
        
        if traverse_visits[state][0]==0:
            if combinatorial:
                val=comb_roll_out(dummy_board, play_counter%2+1, turn, counter)
            else:
                val=roll_out(dummy_board, play_counter%2+1, turn, counter)
            backprop(visits, scores, val, state_save)
        elif winning_move(dummy_board, counter):
            if combinatorial:
                win_val=1 - (0.5/34)*(abs(np.count_nonzero(dummy_board == 0)-1))
            else:
                win_val=1
            backprop(visits, scores, win_val, state_save)
        elif winning_move(dummy_board, counter%2 +1):
            if combinatorial:
                lose_val=0.5- (0.5/36)*(np.count_nonzero(dummy_board == 0)+1)
            else:
                lose_val=0
            backprop(visits, scores, lose_val, state_save)
        elif len(get_valid_locations(dummy_board))==0:
            if combinatorial:
                draw_val=max((turn - (0.5/34)*(abs(np.count_nonzero(board == 0)-1))),0)
            else:
                draw_val=turn
            backprop(visits, scores, draw_val, state_save)
        else:
            use_state_save=state_save.copy()
            expand(visits, scores, dummy_board, counter, use_state_save)
            col=random.choice(get_valid_locations(dummy_board))
            state=(get_valid_locations(dummy_board)).index(col) + 1
            state_save.append(state)
            row=counter_drop_location(dummy_board, col)
            drop_counter(dummy_board, row, col, play_counter)
            if combinatorial:
                val=comb_roll_out(dummy_board, play_counter%2+1, turn, counter)
            else:
                val=roll_out(dummy_board, play_counter%2+1, turn, counter)
            backprop(visits, scores, val, state_save)
    best_visits=-10000
    best_col=-1
    for state in range(1, len(visits[0])):
        if visits[0][state][0]>best_visits:
            if get_valid_locations(board)[state-1] in two_turn_ai_MCTS(board, counter):
                best_visits=visits[0][state][0]
                best_col=get_valid_locations(board)[state-1]
    return best_col
    

In [ ]:
def expand(visits, scores, board, counter, state_save):
    if winning_move(board, 1) or winning_move(board, 2) or len(get_valid_locations(board))==0:
        return visits, scores
    if len(state_save)!=0:
        state=state_save[0]
        state_save.pop(0)
        expand(visits[state], scores[state], board, counter, state_save)
    else:
        for col in get_valid_locations(board):
            visits.append([0])
            scores.append([0])
        return visits, scores

In [ ]:
def best_ucb1(scores, visits, C):
    best_val=-1000000000000
    for state in range(1,len(visits)):
        if visits[state][0]==0:
            return state
        else:
            ucb1_val=(scores[state][0]/visits[state][0])+C*(math.sqrt(math.log(visits[0])/visits[state][0]))
            if ucb1_val>=best_val:
                best_val=ucb1_val
                stored_state=state
    return stored_state

In [ ]:
def roll_out(board, play_counter, turn, orig_counter):
    if winning_move(board, orig_counter):
        return 1
    elif winning_move(board, orig_counter%2 +1):
        return 0
    elif len(get_valid_locations(board))==0:
        if turn==0:
            return 0
        else:
            return 1
    else:
        dummy_board=board.copy()
        game_over=False
        player_turn=0
        while not game_over:
            if player_turn==0:
                col=random.choice(get_valid_locations(dummy_board))
                row=counter_drop_location(dummy_board, col)
                drop_counter(dummy_board, row, col, play_counter)
                if winning_move(dummy_board, play_counter):
                    if play_counter==orig_counter:
                        return 1
                    else:
                        return 0
                elif len(get_valid_locations(dummy_board))==0:
                    if turn==0:
                        return 0
                    else:
                        return 1
                player_turn+=1
                player_turn%=2
            else:
                col=random.choice(get_valid_locations(dummy_board))
                row=counter_drop_location(dummy_board, col)
                drop_counter(dummy_board, row, col, play_counter%2 + 1)
                if winning_move(dummy_board, play_counter%2 + 1):
                    if play_counter==orig_counter:
                        return 0
                    else:
                        return 1
                elif len(get_valid_locations(dummy_board))==0:
                    if turn==0:
                        return 0
                    else:
                        return 1
                player_turn+=1
                player_turn%=2


In [ ]:
def backprop(visits, scores, val, state_save):
    if len(state_save)!=0:
        state=state_save[0]
        state_save.pop(0)
        visits[state][0]+=1
        scores[state][0]+=val
        backprop(visits[state], scores[state], val, state_save)
    else:
        return visits, scores

In [ ]:
def comb_roll_out(board, play_counter, turn, orig_counter):
    if winning_move(board, orig_counter):
        return 1 - (0.5/34)*(abs(np.count_nonzero(board == 0)-1))
    elif winning_move(board, orig_counter%2 +1):
        return 0.5- (0.5/36)*(np.count_nonzero(board == 0)+1)
    elif len(get_valid_locations(board))==0:
        if turn==0:
            return 0.5
        else:
            return 1 - 2*(0.5/34)
    else:
        dummy_board=board.copy()
        game_over=False
        player_turn=0
        while not game_over:
            if player_turn==0:
                col=random.choice(get_valid_locations(dummy_board))
                row=counter_drop_location(dummy_board, col)
                drop_counter(dummy_board, row, col, play_counter)
                if winning_move(dummy_board, play_counter):
                    if play_counter==orig_counter:
                        return 1 - (0.5/34)*(abs(np.count_nonzero(dummy_board == 0)-1))
                    else:
                        return 0.5- (0.5/36)*(np.count_nonzero(dummy_board == 0)+1)
                elif len(get_valid_locations(dummy_board))==0:
                    if turn==0:
                        return 0.5
                    else:
                        return 1 - 2*(0.5/34)
                player_turn+=1
                player_turn%=2
            else:
                col=random.choice(get_valid_locations(dummy_board))
                row=counter_drop_location(dummy_board, col)
                drop_counter(dummy_board, row, col, play_counter%2 + 1)
                if winning_move(dummy_board, play_counter%2 + 1):
                    if play_counter==orig_counter:
                        return 1 - (0.5/34)*(abs(np.count_nonzero(dummy_board == 0)-1))
                    else:
                        return 0.5 + (0.5/36)*(np.count_nonzero(dummy_board != 0)-6)
                elif len(get_valid_locations(dummy_board))==0:
                    if turn==0:
                        return 0.5
                    else:
                        return 1-2*(0.5/34)
                player_turn+=1
                player_turn%=2


In [ ]:
#simulated game between monte carlo tree search AI and minimax AI
monte=0
minimax_wins=0
draws=0
games=0
first=0
second=0
total_moves=[]
while games!=100:
    game_over = False
    player_turn = games%2
    board=create_board()
    moves=0
    while not game_over:
        if player_turn == 0:
            #ask which column they want to drop the counter in
            dummy_board=board.copy()
            turn1=np.count_nonzero(dummy_board==2)-np.count_nonzero(dummy_board==1)
            player1_col = montecarlo(dummy_board, 1, 7, turn1, True)
            if valid_drop(board,player1_col):
                player1_row = counter_drop_location(board,player1_col)
                drop_counter(board, player1_row, player1_col, 1)
                moves+=1
                player_turn+=1
                player_turn%=2
                if winning_move(board, 1):
                    games+=1
                    monte+=1
                    if turn1==0:
                        first+=1
                    else:
                        second+=1
                    print_board(board)
                    total_moves.append(moves)
                    game_over = True
                elif len(get_valid_locations(board))==0:
                    games+=1
                    draws+=1
                    print_board(board)
                    if turn1==0:
                        minimax_wins+=1
                    else:
                        monte+=1
                    total_moves.append(moves)
                    game_over = True
        else:
            dummy_board = board.copy()
            turn2=np.count_nonzero(dummy_board==1)-np.count_nonzero(dummy_board==2)
            player2_col, score = minimax(dummy_board, 3, -math.inf, math.inf, True)
            if valid_drop(board, player2_col):
                player2_row = counter_drop_location(board,player2_col)
                drop_counter(board, player2_row, player2_col, 2)
                moves+=1
                player_turn+=1
                player_turn%=2
                if winning_move(board,2):
                    games+=1
                    minimax_wins+=1
                    print_board(board)
                    total_moves.append(moves)
                    game_over = True
                elif len(get_valid_locations(board))==0:
                    games+=1
                    draws+=1
                    if turn2==0:
                        monte+=1
                    else:
                        minimax_wins+=1
                    print_board(board)
                    total_moves.append(moves)
                    game_over = True
    print('Total games: ' + str(games))
    print('Monte-Carlo wins: ' + str(monte))
    print('Minimax Wins: ' + str(minimax_wins))
    print('Wins going first: ' + str(first))
    print('Wins going second: ' + str(second))
    print('Draws: ' + str(draws))
average_moves=sum(total_moves)/len(total_moves)
print('Average Moves: ' + str(average_moves))

In [ ]:
#play game against monte carlo tree seach AI
game_over = False
player_turn = int(input('Would you like to go first(enter 1) or second (enter 2): '))-1
board=create_board()
print_board(board)
while not game_over:
    if player_turn == 0:
        player_col = int(input('Which collumn would you like to place your counter? (Enter 0-6)'))
        if valid_drop(board,player_col):
            player_row = counter_drop_location(board,player_col)
            drop_counter(board, player_row, player_col, 1)
            if winning_move(board, 1):
                print('You win!')
                print_board(board)
                game_over = True
            elif len(get_valid_locations(board))==0:
                print('Draw')
                print_board(board)
                game_over = True
            player_turn+=1
            player_turn%=2
    else:
        dummy_board=board.copy()
        turn=np.count_nonzero(dummy_board==1)-np.count_nonzero(dummy_board==2)
        ai_col = montecarlo(dummy_board, 2, 7, turn, True)
        if valid_drop(board,ai_col):
            ai_row = counter_drop_location(board,ai_col)
            drop_counter(board, ai_row, ai_col, 2)
            if winning_move(board, 2):
                print('You lose :(')
                print_board(board)
                game_over = True
            elif len(get_valid_locations(board))==0:
                print('Draw')
                print_board(board)
                game_over = True
            player_turn+=1
            player_turn%=2
    print_board(board)